# 1. Dependencies

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import shutil
import numpy as np
import pandas as pd
import pickle
import cv2 as cv
import matplotlib.pyplot as plt
import praw
import requests
import json
import csv
import time
import datetime


# 2. Data Acquisation

##### We mainly (not limited to) need the following  data about posts to model a flair detector
- ID
- Title
- Body
- Comments
- URLs
- Flair
- Other Stats

##### Ref: https://praw.readthedocs.io/en/latest/index.html

In [3]:
# creating a reddit app instance to collect post data from r/india submreddit
reddit = praw.Reddit(client_id='xVeb5-ej49aBDg', client_secret='HLuuBB0e1upw1pbZ-oUFtrBplFY', user_agent='reddit-scrap', username='macabdul9', password='Sudo$0#1')

In [4]:
# pushshit.io api query params to colllect the data

def getPushshiftSubData(ids):
    # url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    # print(url)
    url = "https://api.pushshift.io/reddit/comment/search?ids="+str(ids)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [5]:
# getPushshiftSubData()

In [6]:
subm = reddit.submission(id="fxqifi")

In [7]:
import time

##### Turn Around Time(TAT) for getting all comments of a post is way higher using praw with submission_id than pushshift.io api comment search below code snippets illustrate the same

In [8]:
# start_time = time.time()
# comment_ids = []
# count = 0
# for comment in subm.comments.list():
#     comment_ids.append(comment.id)
#     print(comment.body)
#     count+=1
# print("--- %s seconds ---" % (time.time() - start_time))
# print(count)

In [9]:
# start_time = time.time()
# comments = getPushshiftSubData(','.join(comment_ids))
# count = 0
# for comment in comments:
#     print(comment['body'])
#     count +=1
# print("--- %s seconds ---" % (time.time() - start_time))
# print(count)

In [24]:
comment_list = subm.comments.list()

In [31]:
subm.over_18

False

In [8]:
# for comment in subm.comments.list():
#     # print(comment)
#     # print(comment.score)
#     # break

In [14]:
def get_comments(url):
    # get top 20 comments if total comments are more than that
    subm = reddit.submission(url=url)

    # comments = subm.comments.list()
    # if len(comments) > 10:
    #     comments = comments[:10]

    comments_body = []
    sum_score = 0
    try:
        for i, comment in subm.comments.list():
            
            # There exist some comments which has no body
            try : 
                comments_body.append(comment.body)
            except:
                comments_body.append('')
            
            # There exist some comments which has not a score
            try:
                sum_score += comment.score
            except:
                sum_score += 0            
            
            # We only need 10 comments
            if (i+1)%10==0:
                break
        try:
            mean_score = sum_score/10
        except:
            mean_score = 0
    except:
        return comments_body, 0.0

    return " ".join(comments_body), mean_score

In [126]:
# get_comments('https://www.reddit.com/r/india/comments/d0ytzc/nasa_on_twitter_space_is_hard_we_commend_isro_s/')

In [15]:
def getPushshiftSubData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=2000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [18]:
def collectSubData(subm):
    subData = list() #list to store data points
    
    sub_id = subm['id']
    
    sub_text = subm['title']
    
    author = subm['author']
    
    full_link = subm['full_link']
    
    permalink = subm['permalink']
    
    url = subm['url']
    
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    
    
    score = subm['score']
    
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    
    numComms = subm['num_comments']
    
    
    over_18 = subm['over_18']    
    
            
    comments, mean_comment_score = get_comments(subm['full_link'])    
    
    subData.append((sub_id, sub_text, author, full_link, permalink, url, flair, score, created, numComms, comments, mean_comment_score, over_18))
    
    subStats[sub_id] = subData

In [19]:
#Subreddit to query
sub='india'

#before and after dates
after = "1577836800"  # Wed, 01 Jan 2020 00:00:00 
before = "1585699200" # Wed, 01 Apr 2020 00:00:00 
query = "" # title should have either null string more 
subCount = 0
subStats = {}

In [ ]:
data = getPushshiftSubData(query, after, before, sub)# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    # print(len(data))
    if (subCount + 1) % 1000 == 0:
        print(f'{subCount} submission collected')
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftSubData(query, after, before, sub)

print(len(data))

https://api.pushshift.io/reddit/search/submission/?title=&size=2000&after=1577836800&before=1585699200&subreddit=india
2020-01-03 14:25:07
https://api.pushshift.io/reddit/search/submission/?title=&size=2000&after=1578041707&before=1585699200&subreddit=india
2020-01-05 18:32:25
https://api.pushshift.io/reddit/search/submission/?title=&size=2000&after=1578229345&before=1585699200&subreddit=india
2020-01-07 11:46:09
https://api.pushshift.io/reddit/search/submission/?title=&size=2000&after=1578377769&before=1585699200&subreddit=india


In [139]:
len(subStats)

1487

'https://www.reddit.com/r/india/comments/eib2wt/sunny_leones_popularity_in_the_subcontinent/'

In [88]:
subm.num_comments

0

In [89]:
subm.title

"Sunny Leone's popularity in the Subcontinent"

In [92]:
for comment in subm.comments.list():
    print(comment)